<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/AnalyseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_io

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob

path = '/content/drive/My Drive/dsr_project/data/HarvardWordList/'
save_path = path + 'datasets/'

ir_base_path = '/content/drive/My Drive/dsr_project/data/samplicity-ir/'
irpaths0 = glob.glob(ir_base_path + '*/*M-to-S.wav')
irpaths0 = [irpath.split('/')[-1] for irpath in irpaths0]
#irpaths1 = glob.glob(save_path + 'IR_[0-5]*.wav')
#irpaths1 = [irpath.split('/')[-1] for irpath in irpaths1]
#irpaths1 = [irpath.split('_')[2:][0] for irpath in irpaths1]
#print(irpaths0)
#print(irpaths1)

irpaths = []
for i in range(len(irpaths0)):
  irpaths.append(save_path + 'IR_' +str(i) + '_' + irpaths0[i])

irpaths = irpaths[0:6]
irpaths

In [ ]:
import glob, os, shutil

sample_rate = 2**14

# load data
#path = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/'
#save_path = path + 'datasets_M/'
path = '/content/drive/My Drive/dsr_project/data/HarvardWordList/'
save_path = path + 'datasets/'

ir_length = len(irpaths)

dataset_filenames = glob.glob(save_path + 'tf_IR_*')
dataset_filenames = dataset_filenames[:ir_length]

datasets = []
for filename in dataset_filenames:
  # load the tensorflow dataset
  datasets.append(tf.data.experimental.load(filename))

dataset = datasets[0]
for i in range(1, len(datasets)):
#for i in range(1, len(irpaths)):
  dataset = dataset.concatenate(datasets[i])

# determine size of the dataset
dataset_size = sum(1 for _ in dataset)
print(f"dataset_size: {dataset_size}")

In [ ]:
def lambda_1(features, labels, targets):
  return (features, labels)

def lambda_2(features, labels, targets):
  return (features, targets)

def lambda_3(features, labels, targets):
  return labels

# obtain the datasets containing only labels or targets
label_dataset = dataset.map(lambda features, labels, targets: lambda_1(features, labels, targets))
target_dataset = dataset.map(lambda features, labels, targets: lambda_2(features, labels, targets))

# obtain all labels
all_labels = dataset.map(lambda features, labels, targets: lambda_3(features, labels, targets))

# transform all_labels dataset to numpy array
all_labels_np = []
for label in all_labels:
  all_labels_np.append(tfds.as_numpy(label))

all_labels_np = np.array(all_labels_np)

# determine unique labels
unique_labels, counts = np.unique(all_labels_np, return_counts=True)
print(f"unique_labels: {unique_labels}")
#print(counts)

# determine number of unique labels
nr_of_unique_labels = unique_labels.shape[0]
print(f"nr_of_unique_labels: {nr_of_unique_labels}")

#dataset = dataset.shuffle(dataset_size)

train_fraction = 0.9
validate_dataset_size = int(dataset_size * (1.0-train_fraction)) # 20 percent of dataset_size
#total_dataset = dataset.take(validate_dataset_size)
total_dataset = dataset

In [ ]:
import tensorflow_io as tfio

def preprocess(sample, target):
  audio = tf.cast(sample, tf.float32)
  label = target
  #print(audio)
  #print(label)
  spectrogram = tfio.audio.spectrogram(audio, nfft=1024, window=1024, stride=64)
  spectrogram = tfio.audio.melscale(spectrogram, rate=sample_rate, mels=128, fmin=0, fmax=2000)
  spectrogram /= tf.math.reduce_max(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=-1)

  spectrogram = tf.image.resize(spectrogram, (128, 128))
  spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2))
  spectrogram = spectrogram[::-1, :, :]

  return spectrogram, label

dataset_total = total_dataset.map(lambda feature, label, target: preprocess(feature, label))
dataset_total = dataset_total.cache()
dataset_total = dataset_total.batch(32)

In [ ]:
model_save_path = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/' + 'datasets_F/'
model = tf.keras.models.load_model(model_save_path + 'my_model_2.h5')
model.summary()
model.evaluate(dataset_total)

In [ ]:
X_total = []
y_total = []
for sample, label in dataset_total:
  X_total.append(sample.numpy())
  y_total.append(label.numpy())

probabilities = []
for X in X_total:
  probabilities.extend(model.predict(X))

probabilities = np.asarray(probabilities)  

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio, display

max_probabilities = []
for probs in probabilities:
  max_probabilities.append(np.max(probs))

min_max_probabilities = np.min(max_probabilities)
print(min_max_probabilities)
binned_probs, bins, patches = plt.hist(max_probabilities, bins=10)
plt.show()
print(binned_probs)
print(bins)

In [ ]:
samples_total = []
targets_total = []
lables_total = []
for sample, lable, target in total_dataset:
  samples_total.append(sample.numpy())
  targets_total.append(target.numpy())
  lables_total.append(lable.numpy())

samples_total = np.asarray(samples_total)
targets_total = np.asarray(targets_total)
lables_total = np.asarray(lables_total)

print(samples_total.shape)
print(targets_total.shape)

unique_elements, counts_elements = np.unique(lables_total, return_counts=True)
print(unique_elements)
print(counts_elements)

In [ ]:
from numpy.fft import fft, ifft
import librosa

#irpaths = glob.glob(save_path + 'IR_*.wav')

def wiener_deconvolution(signal, kernel, lambd=1e-3):
  kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
  H = fft(kernel)
  Y = fft(signal)
  #S = np.abs(fft(signal))**2
  #GY = Y*np.conj(H)*S/(H*np.conj(H)*S + lambd**2)
  GY = Y*np.conj(H)/(H*np.conj(H) + lambd**2)
  deconvolved = np.real(ifft(GY))
  return deconvolved

def my_conv(signal, kernel):
  kernel_padded = np.hstack((kernel, np.zeros(len(signal) - len(kernel))))
  convolved_signal = np.real(ifft(fft(kernel_padded)*fft(signal)))
  return convolved_signal

irs = []
#for irpath in irpaths:
for i in range(len(irpaths)):
  ir, IR_sample_rate = librosa.load(irpaths[i], sr=sample_rate)
  irs.append(ir)

all_lengths = []
for ir in irs:
  all_lengths.append(len(ir))

max_length = max(all_lengths)    
min_length = min(all_lengths)
print(max_length/sample_rate)
print(min_length/sample_rate)

In [ ]:
i00 = 3*2*301
i0, i1 = i00 + 0, i00 + 10
test_target = np.concatenate(targets_total[i0:i1])
test_sample = np.concatenate(samples_total[i0:i1])
test_probabilities = probabilities[i0:i1]
mean_test_probabilities = np.mean(test_probabilities, axis = 0)
print(f"mean_test_probabilities: {mean_test_probabilities}")
most_probable_ir = np.argmax(mean_test_probabilities)
print(f"most_probable_ir: {most_probable_ir}")
test_deconv = wiener_deconvolution(test_sample, irs[most_probable_ir])
display(Audio(test_target, rate=sample_rate))
display(Audio(test_sample, rate=sample_rate))
display(Audio(test_deconv, rate=sample_rate))

In [ ]:
probabilities.shape

In [ ]:
probabilities2 = probabilities.reshape((6,-1,6))
probabilities2.shape

In [ ]:
targets_total.shape